# 编辑一个简单的表单

更新一下在上一个教程中编写的投票详细页面的模板（polls/detail.html），让它包含一个HTML<form> 元素：

In [ ]:
<h1>{{ question.question_text }}</h1>

{% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}
#如果有错误信息error_message，就显示出来

<form action="{% url 'polls:vote' question.id %}" method="post">  
{% csrf_token %}  #所有针对内部URL的POST表单都应该使用{% csrf_token %}模板标签，用于防止跨站点请求伪造
{% for choice in question.choice_set.all %}  #列出问题的每个选项
    <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}" />  #单选按钮
    <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br />
{% endfor %}
<input type="submit" value="投票！" />  #提交按钮
</form>

### 编写view()函数

在教程中，我们在polls/urls.py中为投票应用创建了一个URLconf ，包含这一行：

In [ ]:
url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote')

在其中创建了一个vote()函数的虚拟实现。现在让我们来创建一个真实的版本。 将下面的代码添加到polls/views.py

In [ ]:
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect, HttpResponse
from django.core.urlresolvers import reverse

from .models import Choice, Question
# ...
def vote(request, question_id):
    p = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = p.choice_set.get(pk=request.POST['choice'])
        #request.POST 是一个类字典对象，让你可以通过关键字的名字获取提交的数据。 request.POST['choice'] 以字符串形式返回选择的Choice的ID。
        
    except (KeyError, Choice.DoesNotExist):
        # 重新显示页面，给error_message赋值
        return render(request, 'polls/detail.html', {
            'question': p,
            'error_message': "You didn't select a choice.",
        })
    else:
        selected_choice.votes += 1
        selected_choice.save()
        # 投票成功后，重链接至结果页面
        return HttpResponseRedirect(reverse('polls:results', args=(p.id,)))
        # reverse() 调用将返回一个这样的字符串：'/polls/3/results/'

### 在views.py编写results视图

In [ ]:
from django.shortcuts import get_object_or_404, render


def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/results.html', {'question': question})

创建一个polls/results.html模板：

In [ ]:
<h1>{{ question.question_text }}</h1>

<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
{% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">Vote again?</a>

## 使用通用视图

### 改良URLconf 

 polls/urls.py  #注意在第二个和第三个模式的正则表达式中，匹配的模式的名字变成 pk

In [ ]:
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    url(r'^(?P<pk>[0-9]+)/$', views.DetailView.as_view(), name='detail'),
    url(r'^(?P<pk>[0-9]+)/results/$', views.ResultsView.as_view(), name='results'),
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]

### 改良视图

删除旧的index、detail和 results 视图，并用Django的通用视图代替。打开polls/views.py文件，并将它修改成：

In [ ]:
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from django.views import generic

from .models import Choice, Question


class IndexView(generic.ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = 'polls/detail.html'


class ResultsView(generic.DetailView):
    model = Question
    template_name = 'polls/results.html'


def vote(request, question_id):
    ... # same as above

我们在这里使用两个通用视图：ListView 和 DetailView。这两个视图分别抽象“显示一个对象列表”和“显示一个特定类型对象的详细信息页面”这两种概念。

    每个通用视图需要知道它将作用于哪个模型。 这由model 属性提供。
    DetailView期望从URL中捕获名为"pk"的主键值，所以我们为通用视图把question_id改成pk 。
